In [1]:
#Download all dependencies, only need to do this once
#import sys
#!{sys.executable} -m pip install spacy
#!{sys.executable} -m spacy download en_core_web_lg
#!{sys.executable} -m pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bc5cdr_md-0.4.0.tar.gz
#!{sys.executable} -m pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bionlp13cg_md-0.4.0.tar.gz

import nltk
#nltk.download('stopwords')
#nltk.download('punkt')
from nltk.corpus import stopwords
from nltk import tokenize
STOPWORDS = set(stopwords.words('english'))
import csv
import spacy
import numpy as np
import pandas as pd

In [2]:
# load spaCy models
nlp = spacy.load('en_core_web_lg')
nlpSci = spacy.load("en_ner_bc5cdr_md")
nlpSci2 = spacy.load('en_ner_bionlp13cg_md')

In [3]:
abstracts = []
pmids = []

with open("orphanet_epi_data_test.csv", newline='', encoding ='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        abstract = row[4]
        # Remove stopwords
        for word in STOPWORDS:
            token = ' ' + word + ' '
            abstract = abstract.replace(token, ' ')
            abstract = abstract.replace(' ', ' ')
        # Only keep the article if the abstract has more than 5 chars, and it's not one of the curator articles
        if len(abstract)>5:
            abstracts.append(abstract)
            pmids.append(row[3])

In [6]:
for i in range(7,9):
    print(pmids[i])
    print(abstracts[i])

10480370
We calculated relative frequency birth prevalence lysosomal storage diseases (LSDs) The Netherlands based 963 enzymatically confirmed cases diagnosed period 1970-1996. The combined birth prevalence LSDs 14 per 100,000 live births. Glycogenosis type II frequent LSD birth prevalence 2.0 per 100,000 live births, representing 17% diagnosed cases. Within group lipidoses, metachromatic leukodystrophy (MLD) frequent LSD. MLD diagnosed 24% lipidoses calculated birth prevalence 1.42 per 100,000 types combined. Krabbe disease, diagnosed 17% cases, also belongs frequent lipid storage diseases The Netherlands birth prevalence 1.35 per 100,000. The birth prevalence Gaucher disease, commonly regarded frequent lipid storage disease 1.16 per 100,000 types combined. The combined birth prevalence lipid storage diseases 6.2 per 100,000 live births. Within group mucopolysaccharidoses (MPSs), MPS I highest calculated birth prevalence 1.19 per 100,000 (25% cases MPS diagnosed), slightly frequent MP

In [7]:
# Standardize the text of the abstract by replacing all named entities with their entity label
# Eg: 3 patients were seen in a clinic in England --> CARDINAL patients were seen in a clinic in GPE
def standardizeAbstract(abstract):
    doc = nlp(abstract)
    newAbstract = abstract
    # interate through the entities in the abstract
    for e in reversed(doc.ents):
        # replace entities with their label
        #GPE = GeoPolitical Entity
        if e.label_ in {'PERCENT','CARDINAL','GPE','LOC','DATE','TIME','QUANTITY','ORDINAL'}:
            start = e.start_char
            end = start + len(e.text)
            newAbstract = newAbstract[:start] + e.label_ + newAbstract[end:]
    return newAbstract

In [8]:
# Same process but include scientific entities from scispaCy models
def standardizeSciTerms(abstract):
    doc = nlpSci(abstract)
    newAbstract = abstract
    for e in reversed(doc.ents):
        start = e.start_char
        end = start + len(e.text)
        newAbstract = newAbstract[:start] + e.label_ + newAbstract[end:]
        
    doc = nlpSci2(newAbstract)
    for e in reversed(doc.ents):
        start = e.start_char
        end = start + len(e.text)
        newAbstract = newAbstract[:start] + e.label_ + newAbstract[end:]
    return newAbstract

In [9]:
#Turn abstract list into abstract list of sentences
#Tokenize abstract into list of sentences -shows as list within list
for i in range(len(abstracts)):
    abstracts[i] = tokenize.sent_tokenize(abstracts[i])

In [10]:
print(abstracts[7])
#print(abstracts[2])
print('3rd sentence')
print(abstracts[7][0])

['We calculated relative frequency birth prevalence lysosomal storage diseases (LSDs) The Netherlands based 963 enzymatically confirmed cases diagnosed period 1970-1996.', 'The combined birth prevalence LSDs 14 per 100,000 live births.', 'Glycogenosis type II frequent LSD birth prevalence 2.0 per 100,000 live births, representing 17% diagnosed cases.', 'Within group lipidoses, metachromatic leukodystrophy (MLD) frequent LSD.', 'MLD diagnosed 24% lipidoses calculated birth prevalence 1.42 per 100,000 types combined.', 'Krabbe disease, diagnosed 17% cases, also belongs frequent lipid storage diseases The Netherlands birth prevalence 1.35 per 100,000.', 'The birth prevalence Gaucher disease, commonly regarded frequent lipid storage disease 1.16 per 100,000 types combined.', 'The combined birth prevalence lipid storage diseases 6.2 per 100,000 live births.', 'Within group mucopolysaccharidoses (MPSs), MPS I highest calculated birth prevalence 1.19 per 100,000 (25% cases MPS diagnosed), sli

In [11]:
def isGeo(sentence):
    doc = nlp(sentence)
    # interate through the entities in the abstract
    for ent in doc.ents:
        if ent.label_ in {'GPE','LOC'}:
            return True
    return False

In [12]:
def whereGeo(sentence):
    locations = []
    doc = nlp(sentence)
    # interate through the entities in the sentence
    for ent in doc.ents:
        if ent.label_ in {'GPE','LOC'}:
            locations.append(ent.text)
    return locations

In [15]:
print(abstracts[5][8])
location = whereGeo(abstracts[5][8])
print(location)
#abstracts[5][8] = whereGeo(abstracts[5][8])
#print(abstracts[5][8][0])


However, group, relatively common represent important health problem Australia.
['Australia']


In [16]:
geopmids = []
geosentences = []
#Returns all of the sentences with location information
i=1
for abstract in abstracts:
    for sentence in abstract:
        if i%50==0:
            print(i)
        if isGeo(sentence):
            geopmids.append(pmids[i])
            geosentences.append(sentence)
    i+=1

In [17]:
for i in range(len(geosentences)):
    print(geopmids[1-i])
    print(geosentences[1-i])

14685153
We calculated relative frequency birth prevalence lysosomal storage diseases (LSDs) The Netherlands based 963 enzymatically confirmed cases diagnosed period 1970-1996.
7900112
However, group, relatively common represent important health problem Australia.
15523498
Salla disease, rare elsewhere, fourth common, stemming founder mutation Salla region northern Finland brought Sweden immigration.<h4>Conclusion</h4>The collective incidence LSDs Sweden essentially equal European countries, somewhat different disease pattern.
15523498
Gaucher disease frequent Sweden European countries, due founder effect mutation (p.L444P) northern Sweden.
25274184
Relatively higher prevalence LSDs reported north Portugal (25 per 100,000).
25274184
The overall prevalence LSDs Czech population (12.25 per 100,000) comparable reported countries well-established advanced diagnostics LSDs Netherlands (14 per 100,000), Australia (12.9 per 100,000) Italy (12.1 per 100,000).
25274184
The aim retrospective stu

In [18]:
print(geopmids)
print(geosentences)

['7900112', '14685153', '14685153', '20490927', '25274184', '25274184', '25274184', '15523498', '15523498']
['However, group, relatively common represent important health problem Australia.', 'We calculated relative frequency birth prevalence lysosomal storage diseases (LSDs) The Netherlands based 963 enzymatically confirmed cases diagnosed period 1970-1996.', 'Krabbe disease, diagnosed 17% cases, also belongs frequent lipid storage diseases The Netherlands birth prevalence 1.35 per 100,000.', 'The overall birth prevalence 29 different LSDs studied Portuguese population calculated 25/100000 live births, twice prevalence previously described Australia The Netherlands.', 'The aim retrospective study determine prevalence lysosomal storage disorders (LSDs) Czech Republic.', 'The overall prevalence LSDs Czech population (12.25 per 100,000) comparable reported countries well-established advanced diagnostics LSDs Netherlands (14 per 100,000), Australia (12.9 per 100,000) Italy (12.1 per 100,0

In [19]:
#Returns dictionary of pubmed id and 
locInfo = {}

i=0
for abstract in abstracts:
    for sentence in abstract:
        if isGeo(sentence):
            if(i>15):
                break
            print(i)
            print(pmids[i])
            print(sentence)
            print(whereGeo(sentence))
            print('............................')
            locInfo[pmids[i]]=whereGeo(sentence)
    i+=1

5
9918480
However, group, relatively common represent important health problem Australia.
['Australia']
............................
7
10480370
We calculated relative frequency birth prevalence lysosomal storage diseases (LSDs) The Netherlands based 963 enzymatically confirmed cases diagnosed period 1970-1996.
['Netherlands']
............................
7
10480370
Krabbe disease, diagnosed 17% cases, also belongs frequent lipid storage diseases The Netherlands birth prevalence 1.35 per 100,000.
['Netherlands']
............................
8
14685153
The overall birth prevalence 29 different LSDs studied Portuguese population calculated 25/100000 live births, twice prevalence previously described Australia The Netherlands.
['Australia', 'Netherlands']
............................
9
20490927
The aim retrospective study determine prevalence lysosomal storage disorders (LSDs) Czech Republic.
['Czech Republic']
............................
9
20490927
The overall prevalence LSDs Czech popul

In [20]:
print(locInfo)

{'9918480': ['Australia'], '10480370': ['Netherlands'], '14685153': ['Australia', 'Netherlands'], '20490927': ['Portugal'], '25274184': ['Finland', 'Sweden', 'Sweden']}
